# Semantic Kernel (SK) Integration into Azure Machine Learning (AzureML)

**Requirements** - In order to benefit from this tutorial, you will need:
* A basic understanding of Machine Learning and Large Language Models
* An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
* An Azure Machine Learning Workspace and Azure Container Registry
* An OpenAI API Key which can be found in User Settings in OpenAI

**Motivations** - Semantic kernel has a slightly different approach to LLM agents. It offers an interesting Plan->Execute pattern, where it could use LLM to form a plan first, then human could confirm and execute on the plan. In this notebook, we use the [planner](https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/python/05-using-the-planner.ipynb) example from Semantic Kernel as a base. But additionally, we've made the following modifications:
* Created a **python SemanticKernelHttp server** based on Flask.
* Deploy SemanticKernelHttp to an **AzureML Managed Online Endpoint**

Managed online endpoints provide an easy to manage inferencing server for your ML workload. It's perfect for LLM based applications. Since we need a REST service, we won't use the default endpoint docker image, we will create a custom docker image instead.

**Outline** - 
1. Prepare Dependencies
2. Deploy to Managed Online Endpoint
3. Test

# 1. Connect to Azure Machine Learning

In [ ]:
OPENAI_API_TYPE = "openai"  # 'azure' or 'openai'
OPENAI_API_KEY = "<OPENAI-API-KEY>"

# required for OpenAI API
OPENAI_ORG_ID = ""
OPENAI_MODEL_ID = "gpt-3.5-turbo"

# required for Azure OpenAI API
AZURE_OPENAI_API_ENDPOINT = "https://<AZURE_OPENAI_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_DEPLOYMENT_NAME = "<DEPLOYMENT_NAME>"

# set to true for chat completion API, false for text completion
IS_CHAT_COMPLETION = True

# setting up env variables for local server
%env OPENAI_API_TYPE=$OPENAI_API_TYPE
%env OPENAI_API_KEY=$OPENAI_API_KEY
%env OPENAI_MODEL_ID=$OPENAI_MODEL_ID
%env OPENAI_ORG_ID=$OPENAI_ORG_ID
%env AZURE_OPENAI_API_ENDPOINT=$AZURE_OPENAI_API_ENDPOINT
%env AZURE_OPENAI_API_DEPLOYMENT_NAME=$AZURE_OPENAI_API_DEPLOYMENT_NAME
%env IS_CHAT_COMPLETION=$IS_CHAT_COMPLETION

# Install python dependencies
%pip install -r ../src/sk/requirements.txt

### 1.1 Set workspace details

In [ ]:
# enter details of your AML workspace
SUBSCRIPTION_ID = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP = "<RESOURCE_GROUP>"
AML_WORKSPACE_NAME = "<AML_WORKSPACE_NAME>"

### 1.2 Login to your Azure account

In [ ]:
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    AzureCliCredential,
)

try:
    credential = DefaultAzureCredential(additionally_allowed_tenants=["*"])
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential(additionally_allowed_tenants=["*"])

# If login doesn't work above, uncomment the code below and login using device code
# !az login --use-device-code

### 1.3 Create Container Registry and Docker Image

In [ ]:
import re
from azure.ai.ml import (
    MLClient,
)

ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, AML_WORKSPACE_NAME)
ws = ml_client.workspaces.get(AML_WORKSPACE_NAME)

# Get the Azure Container Registry associated with the workspace
acr = ws.container_registry

# Parse the ACR resource Id for the ACR name
match_object = re.match(r".+?registries\/(.+)", acr)
ACR_NAME = match_object.group(1)

In [ ]:
# Build the image in your ACR image
ACR_IMAGE_NAME = "serving"

!az acr build --image {ACR_IMAGE_NAME} --registry {ACR_NAME} ./environment/serving/. --resource-group {RESOURCE_GROUP}

# 2. Managed Online Endpoint
### 2.1 Create Endpoint

In [ ]:
# create a endpoint
import datetime

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
)

from azure.ai.ml import (
    MLClient,
)

time = str(datetime.datetime.now().strftime("%m%d%H%M%f"))
online_endpoint_name = f"aml-llm-sk-demo-{time}"

ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, AML_WORKSPACE_NAME)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="online endpoint for SemanticKernelHttp server",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(endpoint)

# 3. Store the API Key in KeyVault

The below code is modelled after the example notebook [online-endpoints-keyvault.ipynb](../../managed/online-endpoints-keyvault.ipynb).

### 3.1 Import Keyvault Libraries

In [ ]:
from azure.mgmt.keyvault import KeyVaultManagementClient
from azure.keyvault.secrets import SecretClient
from azure.mgmt.keyvault.models import (
    VaultCreateOrUpdateParameters,
    VaultProperties,
    Sku,
)
from azure.mgmt.keyvault.models import AccessPolicyEntry, Permissions, SecretPermissions

### 3.2 Create a Keyvault Management client

In [ ]:
KEYVAULT_NAME = f"llmdemokv{time}"
KV_OPENAI_KEY = "OPENAI-API-KEY"

keyvault_mgmt_client = KeyVaultManagementClient(
    credential=credential, subscription_id=SUBSCRIPTION_ID
)

### 3.3 Get your Object Id
The `oid` in your JWT access token represents the Object ID of the current user or Service Principal logged into the Azure CLI.

In [ ]:
import json, base64

cli_credential = AzureCliCredential()
token = cli_credential.get_token("https://management.azure.com").token
user_or_sp_object_id = json.loads(base64.b64decode(token.split(".")[1] + "===")).get(
    "oid"
)

### 3.4 Define an AccessPolicy for the Endpoint and the current user

Allow the endpoint to get secrets in the keyvault and allow all secret permissions for the current user or Service Principal.

In [ ]:
endpoint_access_policy = AccessPolicyEntry(
    tenant_id=endpoint.identity.tenant_id,
    object_id=endpoint.identity.principal_id,
    permissions=Permissions(secrets=[SecretPermissions.GET]),
)

In [ ]:
user_or_sp_access_policy = AccessPolicyEntry(
    tenant_id=endpoint.identity.tenant_id,
    object_id=user_or_sp_object_id,
    permissions=Permissions(secrets=[SecretPermissions.ALL]),
)

### 3.5 Create the Keyvault

In [ ]:
keyvault = keyvault_mgmt_client.vaults.begin_create_or_update(
    vault_name=KEYVAULT_NAME,
    resource_group_name=RESOURCE_GROUP,
    parameters=VaultCreateOrUpdateParameters(
        location=endpoint.location,
        properties=VaultProperties(
            tenant_id=endpoint.identity.tenant_id,
            sku=Sku(name="Standard", family="A"),
            access_policies=[endpoint_access_policy, user_or_sp_access_policy],
        ),
    ),
).result()

### 3.6 Add your OPENAI_API_KEY to the Keyvault

In [ ]:
KEYVAULT_URL = f"https://{KEYVAULT_NAME}.vault.azure.net"

secret_client = SecretClient(credential=credential, vault_url=KEYVAULT_URL)
secret = secret_client.set_secret(name=KV_OPENAI_KEY, value=OPENAI_API_KEY)

# 4. Deploy to the Endpoint

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    OnlineRequestSettings,
    Model,
    Environment,
)

deployment_name = f"deploy-{time}"
sk_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    model=Model(path="../src"),
    request_settings=OnlineRequestSettings(request_timeout_ms=60000),
    environment=Environment(
        image=f"{ACR_NAME}.azurecr.io/{ACR_IMAGE_NAME}:latest",
        name="serving",
        description="A generic serving environment, allowing customer to provide their own entry point to bring up an http server",
        inference_config={
            "liveness_route": {"port": 5001, "path": "/health"},
            "readiness_route": {"port": 5001, "path": "/health"},
            "scoring_route": {"port": 5001, "path": "/"},
        },
    ),
    environment_variables={
        "AZUREML_SERVING_ENTRYPOINT": "src/sk/entry.sh",
        "OPENAI_API_KEY": f"keyvaultref:{KEYVAULT_URL}/secrets/{KV_OPENAI_KEY}",
        "OPENAI_API_TYPE": OPENAI_API_TYPE,
        "OPENAI_MODEL_ID": OPENAI_MODEL_ID,
        "OPENAI_ORG_ID": OPENAI_ORG_ID,
        "AZURE_OPENAI_API_ENDPOINT": AZURE_OPENAI_API_ENDPOINT,
        "AZURE_OPENAI_API_DEPLOYMENT_NAME": AZURE_OPENAI_API_DEPLOYMENT_NAME,
        "IS_CHAT_COMPLETION": True,
    },
    endpoint_name=online_endpoint_name,
    instance_type="Standard_F2s_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(sk_deployment).result()

endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint).result()

# 5. Test
Now endpoint has been deployed, let's test it.

In [ ]:
import requests, json
from urllib.parse import urlsplit

url_parts = urlsplit(endpoint.scoring_uri)
url = url_parts.scheme + "://" + url_parts.netloc

token = ml_client.online_endpoints.get_keys(name=online_endpoint_name).primary_key
headers = {"Authorization": "Bearer " + token, "Content-Type": "application/json"}
payload = json.dumps(
    {
        "value": "Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French."
    }
)

response = requests.post(f"{url}/planner/createplan", headers=headers, data=payload)
print(f"Created Plan:\n", response.text)

In [ ]:
payload = response.text
response = requests.request(
    "POST", f"{url}/planner/executeplan", headers=headers, data=payload
)
print(f"Execution Result:\n", response.text)

# 6. Clean up resources

### 6.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

### 6.2 Delete the ACR Image

In [ ]:
!az acr repository delete --name {ACR_NAME} --image {ACR_IMAGE_NAME}:latest --yes

### 6.3 Delete the KeyVault

In [ ]:
keyvault_mgmt_client.vaults.delete(RESOURCE_GROUP, KEYVAULT_NAME)

.